In [3]:
# imports
from __future__ import division
import numpy as np
from collections import namedtuple

In [4]:
# Original MINC network on our images

import minc_model.copy_minc_labeled_surface_images
import minc_model.test

minc_model.copy_minc_labeled_surface_images.go()
answers, confusion, categories, net = minc_model.test.classify_images('googlenet')
accuracy = np.sum(np.diag(np.array(confusion.values())))/np.sum(np.array(confusion.values()))

OrigMinc = namedtuple('OrigMinc', ['acc', 'net', 'answers', 'confusion', 'materials'])
orig_minc = OrigMinc(accuracy, net, answers, confusion, categories)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars


In [7]:
orig_minc.net